In [ ]:
pip install requests transformers beautifulsoup4


In [ ]:
from transformers import pipeline
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
import requests

from bs4 import BeautifulSoup

# Function to retrieve news sentiment from the API
def get_news_sentiment(ticker, api_key, limit=50, sort='LATEST'):
    # Base URL for the API
    url = 'https://www.alphavantage.co/query'

    # Parameters for the API request
    params = {
        'function': 'NEWS_SENTIMENT',
        'tickers': ticker,
        'limit': limit,
        'sort': sort,
        'apikey': api_key
    }

    # Send the GET request
    response = requests.get(url, params=params)

    # Check if the response status code is 200 (OK)
    if response.status_code == 200:
        # Parse the JSON data from the response
        data = response.json()

        # Return the data if successful
        return data
    else:
        # If the API call was not successful, return None or error message
        print(f"Error fetching data: {response.status_code}")
        return None

# Function to extract and clean text content from a URL
def get_article_text(article_url):
    response = requests.get(article_url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Find relevant HTML tags (headlines and paragraphs)
    results = soup.find_all(['h1', 'p'])
    text = [result.get_text() for result in results]

    # Combine the text into a single article
    ARTICLE = " ".join(text)
    return ARTICLE

# Function to prepare text for summarization and summarize it
def summarize_text(text, summarizer):
    # Split text into sentences and prepare chunks for summarization
    ARTICLE = text.replace('.', '.<eos>').replace('!', '!<eos>').replace('?', '?<eos>')
    sentences = ARTICLE.split('<eos>')

    # Chunk the article text if it's too large for the summarizer
    max_chunk = 500
    chunks = []
    current_chunk = 0
    for sentence in sentences:
        if len(chunks) == current_chunk + 1:
            if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
                chunks[current_chunk].extend(sentence.split(' '))
            else:
                current_chunk += 1
                chunks.append(sentence.split(' '))
        else:
            chunks.append(sentence.split(' '))

    # Combine the chunks into strings
    for chunk_id in range(len(chunks)):
        chunks[chunk_id] = ' '.join(chunks[chunk_id])

    # Run the summarizer on the chunks
    summarized = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

    # Combine the summaries into one text
    summary_text = ' '.join([summ['summary_text'] for summ in summarized])

    return summary_text

# Function to display and summarize the news articles
def display_and_summarize_news(news_data, summarizer):
    if 'feed' in news_data:
        articles = news_data['feed']
        if articles:
            for i, article in enumerate(articles[:3]):  # Limit to first 3 articles for demo
                print(f"\nArticle {i+1}: {article['title']}")
                print(f"Source: {article['source']}")
                print(f"Published: {article['time_published']}")

                # Fetch the full article text from the URL
                article_url = article['url']
                full_text = get_article_text(article_url)

                # Summarize the article text
                summary = summarize_text(full_text, summarizer)

                # Display the summary
                print(f"Summary: {summary}\n")
        else:
            print("No news articles found for this ticker.")
    else:
        print("Invalid response structure.")

# Main function
if __name__ == "__main__":
    # Input from the user
    ticker = input("Enter the stock ticker symbol (e.g., AAPL for Apple): ")
    api_key = '#'  # Replace with your actual API key

    # Initialize the summarizer from Huggingface's pipeline
    #summarizer = pipeline("summarization")

    # Fetch the news sentiment data
    news_data = get_news_sentiment(ticker, api_key)

    # If the news data is retrieved, summarize and display it
    if news_data:
        display_and_summarize_news(news_data, summarizer)
    else:
        print("Failed to retrieve news data.")


Enter the stock ticker symbol (e.g., AAPL for Apple): nvda

Article 1: Artificial Intelligence  ( AI )  Energy Consumption Is Jumping At a Scary Pace: 2 Stocks That Could Surge Thanks to This Trend
Source: Motley Fool
Published: 20241016T220000
Summary:  Two companies are set to solve two major problems arising out of the rapid adoption of AI . The proliferation of artificial intelligence has increased the demand for more powerful chips that are being deployed in data centers to train complex large language models (LLMs) Nvidia's graphics processing units (GPUs) have been the chips of choice for AI training and inference . The second problem that AI data centers are creating is that of higher heat generation .  Nvidia's chips have been deployed for training popular AI models such as OpenAI's ChatGPT and Meta Platforms' Llama . Cloud service providers have been increasingly looking to get their hands on the company's offerings to train even larger models . As such, Nvidia's AI chips are

In [ ]:
import requests
from bs4 import BeautifulSoup

# Function to retrieve news sentiment from the API, filtering for financial market news
def get_news_sentiment(ticker, api_key, limit=50, sort='LATEST'):
    # Base URL for the API
    url = 'https://www.alphavantage.co/query'

    # Parameters for the API request, including the 'financial_markets' topic for stock-related news
    params = {
        'function': 'NEWS_SENTIMENT',
        'tickers': ticker,
        'limit': limit,
        'sort': sort,
        'topics': 'financial_markets',  # Filter for financial market-related news
        'apikey': api_key
    }

    # Send the GET request
    response = requests.get(url, params=params)

    # Check if the response status code is 200 (OK)
    if response.status_code == 200:
        # Parse the JSON data from the response
        data = response.json()

        # Return the data if successful
        return data
    else:
        # If the API call was not successful, return None or error message
        print(f"Error fetching data: {response.status_code}")
        return None

# Function to extract and clean text content from a URL
def get_article_text(article_url):
    response = requests.get(article_url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Find relevant HTML tags (headlines and paragraphs)
    results = soup.find_all(['h1', 'p'])
    text = [result.get_text() for result in results]

    # Combine the text into a single article
    ARTICLE = " ".join(text)
    return ARTICLE

# Function to prepare text for summarization and summarize it
def summarize_text(text, summarizer):
    # Split text into sentences and prepare chunks for summarization
    ARTICLE = text.replace('.', '.<eos>').replace('!', '!<eos>').replace('?', '?<eos>')
    sentences = ARTICLE.split('<eos>')

    # Chunk the article text if it's too large for the summarizer
    max_chunk = 500
    chunks = []
    current_chunk = 0
    for sentence in sentences:
        if len(chunks) == current_chunk + 1:
            if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
                chunks[current_chunk].extend(sentence.split(' '))
            else:
                current_chunk += 1
                chunks.append(sentence.split(' '))
        else:
            chunks.append(sentence.split(' '))

    # Combine the chunks into strings
    for chunk_id in range(len(chunks)):
        chunks[chunk_id] = ' '.join(chunks[chunk_id])

    # Run the summarizer on the chunks
    summarized = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

    # Combine the summaries into one text
    summary_text = ' '.join([summ['summary_text'] for summ in summarized])

    return summary_text

# Function to display and summarize the news articles
def display_and_summarize_news(news_data, summarizer):
    if 'feed' in news_data:
        articles = news_data['feed']
        if articles:
            for i, article in enumerate(articles[:3]):  # Limit to first 3 articles for demo
                print(f"\n=== Article {i+1} ===")
                print(f"Title: {article['title']}")
                print(f"Source: {article['source']}")
                print(f"Published: {article['time_published']}")

                # Fetch the full article text from the URL
                article_url = article['url']
                full_text = get_article_text(article_url)

                # Summarize the article text
                summary = summarize_text(full_text, summarizer)

                # Display the summary
                print(f"Summary:\n{summary}\n")
                print("="*40)
        else:
            print("No news articles found for this ticker.")
    else:
        print("Invalid response structure.")

# Main function
if __name__ == "__main__":
    # Input from the user
    ticker = input("Enter the stock ticker symbol (e.g., AAPL for Apple): ")
    api_key = 'ENIPDF3XPHW9IUUE'  # Replace with your actual API key

    # Initialize the summarizer from Huggingface's pipeline
    #summarizer = pipeline("summarization")

    # Fetch the news sentiment data, now filtered for stock-related news
    news_data = get_news_sentiment(ticker, api_key)

    # If the news data is retrieved, summarize and display it
    if news_data:
        display_and_summarize_news(news_data, summarizer)
    else:
        print("Failed to retrieve news data.")


Enter the stock ticker symbol (e.g., AAPL for Apple): nvda
No news articles found for this ticker.
